In [4]:
!pip install --upgrade tensorflow --user
!pip install tf-quant-finance
!pip install QuantLib-Python

In [5]:
from QuantLib import *
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tf_quant_finance as tff 
import tensorflow as tf
from scipy.interpolate import interp1d
import time
import pandas as pd

# Shortcut alias
pde = tff.math.pde
option_price = tff.black_scholes.option_price
implied_vol = tff.black_scholes.implied_vol

In [6]:
# tf.function decorator makes the function faster in graph mode.
@tf.function
def american_option(number_grid_points,
                    time_delta,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0,
                    dtype=tf.float64):
  """ Computes American Call options prices.

  Args:
    number_grid_points: A Python int. Number of grid points for the finite
      difference scheme.
    time_delta: A Python float. Grid time discretization parameter.
    strike: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the strikes of the underlying American options. 
    volatility: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the volatilities of the underlying American options. 
    risk_free_rate: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the risk-free interest rates associated with the underlying
      American options.
    expiry: A Python float. Expiry date of the options. If the options
      have different expiries, volatility term has to adjusted to
      make expiries the same.
    dtype: Optional `tf.dtype` used to assert dtype of the input `Tensor`s.

  Returns:
    A tuple of the estimated option prices of shape
    `(number_of_options, number_grid_points)` and the corresponding `Tensor` 
    of grid locations of shape `(number_grid_points,)`.
  """
  # Define the coordinate grid
  s_min = 0.01 #spot/nb_bound #0.01
  s_max = 300. #nb_bound*spot #300.

  grid = pde.grids.uniform_grid(minimums=[s_min],
                                maximums=[s_max],
                                sizes=[number_grid_points],
                                dtype=dtype)

  # Define the values grid for the final condition
  s = grid[0]
  final_values_grid = tf.nn.relu(s - strike)

  # Define the PDE coefficient functions
  def second_order_coeff_fn(t, grid):
    del t
    s = grid[0]
    return [[volatility ** 2 * s ** 2 / 2]]

  def first_order_coeff_fn(t, grid):
    del t
    s = grid[0]
    return [(risk_free_rate-dividend_rates) * s]

  def zeroth_order_coeff_fn(t, grid):
    del t, grid
    return -risk_free_rate

  # Define the boundary conditions
  @pde.boundary_conditions.dirichlet
  def lower_boundary_fn(t, grid):
    del t, grid
    return tf.constant(0.0, dtype=dtype)

  @pde.boundary_conditions.dirichlet
  def upper_boundary_fn(t, grid):
    del grid
    return tf.squeeze(s_max - strike * tf.exp(-(risk_free_rate-dividend_rates) * (expiry - t)))

  # In order to price American option one needs to set option values to 
  # V(x) := max(V(x), max(x - strike, 0)) after each iteration
  def values_transform_fn(t, grid, values):
    del t
    s = grid[0]
    if european_exercise==False:
        values_floor = tf.nn.relu(s - strike)
        return grid, tf.maximum(values, values_floor)
    else:
        return grid, values

  # Solve
  estimate_values, estimate_grid, _, _ = \
    pde.fd_solvers.solve_backward(
      start_time=expiry,
      end_time=0,
      values_transform_fn=values_transform_fn,
      coord_grid=grid,
      values_grid=final_values_grid,
      time_step=time_delta,
      boundary_conditions=[(lower_boundary_fn, upper_boundary_fn)],
      second_order_coeff_fn=second_order_coeff_fn,
      first_order_coeff_fn=first_order_coeff_fn,
      zeroth_order_coeff_fn=zeroth_order_coeff_fn,
      dtype=dtype
    )
  return estimate_values, estimate_grid[0]

def tqf_price(number_grid_points,
                    time_delta,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0,
                    dtype=tf.float64):
    estimate, grid_locations = american_option(
    time_delta=time_delta,
    expiry=tff.datetime.daycount_actual_actual_isda(
          start_date=tff.datetime.dates_from_tuples([(1998, 5, 15)]), 
          end_date=tff.datetime.dates_from_tuples([(1999, 5, 15)]),
          dtype=tf.float64).numpy()[0],
    number_grid_points=number_grid_points,
    volatility=tf.constant(0.20,tf.float64),
    risk_free_rate=tf.constant(0.02,tf.float64),
    dividend_rates=tf.constant(0.07,tf.float64),
    strike=tf.constant(40.0,tf.float64),
    spot=36.0,
    european_exercise=True,
    nb_bound=3.0,
    dtype=tf.float64)

    # Convert to numpy for plotting
    estimate = estimate.numpy()
    grid_locations = grid_locations.numpy()
    tqf_euro_price_pde = interp1d(grid_locations, estimate)(36.0)
    return tqf_euro_price_pde

In [10]:
device = "/cpu:0"
with tf.device(device):

    time_delta = 0.005
    number_grid_points = 1024

    batch_of_options = [5, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 10000]
    for number_of_options in batch_of_options:
        filename = "https://raw.githubusercontent.com/arthurpham/google_colab/main/data/American_Option_Black_Scholes_GenerateCSV_{}.csv".format(number_of_options)
        option_inputs_df = pd.read_csv(filename)
        
        volatilities = option_inputs_df['volatility']
        risk_free_rates = option_inputs_df['risk_free_rate']
        strikes = option_inputs_df['strike']
        expiries = option_inputs_df['expiry']
        t = time.time()
    
        estimate, grid_locations = american_option(
            time_delta=time_delta,
            expiry=tff.datetime.daycount_actual_actual_isda(
                start_date=tff.datetime.dates_from_tuples([(1998, 5, 15)]), 
                end_date=tff.datetime.dates_from_tuples([(1999, 5, 15)]),
                dtype=tf.float64).numpy()[0],
            number_grid_points=number_grid_points,
            volatility=tf.constant(volatilities, dtype=tf.float64, shape=[len(strikes), 1]  ),
            risk_free_rate=tf.constant(risk_free_rates, dtype=tf.float64, shape=[len(strikes), 1]  ),
            dividend_rates=tf.constant(np.random.rand(number_of_options, 1) *0.0, tf.float64 ),
            strike=tf.constant(strikes, dtype=tf.float64, shape=[len(strikes), 1] ),
            spot=36.0,
            european_exercise=True,
            nb_bound=3.0,
            dtype=tf.float64)
        time_tqf = time.time() - t

        tqf_options_per_second = number_of_options / time_tqf
        print('------------------------')
        print('TQF CPU {} options'.format(number_of_options))
        print('wall time: ', time_tqf)
        print('options per second: ', tqf_options_per_second)
        print('------------------------')

------------------------
TQF CPU 5 options
wall time:  0.07661008834838867
options per second:  65.2655558529344
------------------------
------------------------
TQF CPU 10 options
wall time:  0.12093424797058105
options per second:  82.68956203731997
------------------------
------------------------
TQF CPU 100 options
wall time:  2.2292091846466064
options per second:  44.8589574674002
------------------------
------------------------
TQF CPU 150 options
wall time:  1.7748360633850098
options per second:  84.51484793131621
------------------------
------------------------
TQF CPU 200 options
wall time:  2.368826150894165
options per second:  84.43000341097452
------------------------
------------------------
TQF CPU 250 options
wall time:  2.832279682159424
options per second:  88.26811899077414
------------------------
------------------------
TQF CPU 300 options
wall time:  3.3288090229034424
options per second:  90.12232240897227
------------------------
------------------------


In [9]:
device = "/gpu:0"
with tf.device(device):

    time_delta = 0.005
    number_grid_points = 1024

    batch_of_options = [5, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 10000]
    for number_of_options in batch_of_options:
        filename = "https://raw.githubusercontent.com/arthurpham/google_colab/main/data/American_Option_Black_Scholes_GenerateCSV_{}.csv".format(number_of_options)
        option_inputs_df = pd.read_csv(filename)
        
        volatilities = option_inputs_df['volatility']
        risk_free_rates = option_inputs_df['risk_free_rate']
        strikes = option_inputs_df['strike']
        expiries = option_inputs_df['expiry']
        t = time.time()
    
        estimate, grid_locations = american_option(
            time_delta=time_delta,
            expiry=tff.datetime.daycount_actual_actual_isda(
                start_date=tff.datetime.dates_from_tuples([(1998, 5, 15)]), 
                end_date=tff.datetime.dates_from_tuples([(1999, 5, 15)]),
                dtype=tf.float64).numpy()[0],
            number_grid_points=number_grid_points,
            volatility=tf.constant(volatilities, dtype=tf.float64, shape=[len(strikes), 1]  ),
            risk_free_rate=tf.constant(risk_free_rates, dtype=tf.float64, shape=[len(strikes), 1]  ),
            dividend_rates=tf.constant(np.random.rand(number_of_options, 1) *0.0, tf.float64 ),
            strike=tf.constant(strikes, dtype=tf.float64, shape=[len(strikes), 1] ),
            spot=36.0,
            european_exercise=True,
            nb_bound=3.0,
            dtype=tf.float64)
        time_tqf = time.time() - t

        tqf_options_per_second = number_of_options / time_tqf
        print('------------------------')
        print('TQF GPU {} options'.format(number_of_options))
        print('wall time: ', time_tqf)
        print('options per second: ', tqf_options_per_second)
        print('------------------------')

------------------------
TQF GPU 5 options
wall time:  8.331348896026611
options per second:  0.6001429135184341
------------------------
------------------------
TQF GPU 10 options
wall time:  0.40035319328308105
options per second:  24.977944894095593
------------------------
------------------------
TQF GPU 100 options
wall time:  0.45128774642944336
options per second:  221.58811266468658
------------------------
------------------------
TQF GPU 150 options
wall time:  0.44771790504455566
options per second:  335.03239050730485
------------------------
------------------------
TQF GPU 200 options
wall time:  0.49828648567199707
options per second:  401.3755254274593
------------------------
------------------------
TQF GPU 250 options
wall time:  0.5446796417236328
options per second:  458.98539407288604
------------------------
------------------------
TQF GPU 300 options
wall time:  0.6084840297698975
options per second:  493.02855181498705
------------------------
--------------